## "A Bossy Sort of Voice"
#### A study of sexism in the Harry Potter series using Natural Language Processing

My eldest son is almost 6 and loves the Harry Potter series of books.  But, as I read them for the first time with him as a bedtime story, I noticed something hadn't expected.  

The sexism.

The sexism critique of the Harry Potter novels is not a new one - many people have (written)[https://www.bustle.com/articles/136244-the-5-least-feminist-moments-in-harry-potter] excellent articles about Ron's treatment of Hermoine, the portrayl of other female characters as cold or incompetent or promiscuous.  But there were two types of analyses I didn't see: a quantitative or linguistics based one, or something that looked at how the author herself portrays female characters in a biased light in how they speak

My hope in doing this is that people will use some of these tools to look more critically at the language of literature we love and the popular press for signs of gender, racial, and other bias.

#### Getting started: setting up hypotheses and requirements

In this project, I will test three hypotheses:

1. Female characters are referred to by the narrator with sexist words throughout the series by the narrator, while males are not described with that same language.

2. The narrator will use more sexist words when describing the female characters than other characters will use when talking about them.

3. Sexism as defined above will decline in both dialog and narration as the series progresses.


Taking this approach requires tools to do the following, which I'll tackle in this workbook:

1. Process text from all 7 'Harry Potter' books.

2. Seperate dialog from narration.

3. Label dialog and narration.

4. Distinguish parts of speech, like nouns, verbs, and adjectives.
000

2. Look at the text at the word and sentence level.

3. Distinguish parts of speech, like nouns, verbs, and adjectives.

4. 

5. Be able to tell female characters from male characters

6. Have examples of sexist words and/or phrases to create a classifier. 

7. Reduce sexist words in the text to their root forms (e.g. 'shriller' should be equivalent to 'shrill').

8. Summarize the data to test our hypotheses.


#### Step 1: Process text from all 7 'Harry Potter' books.

What I mean by "process", is to get the text from the series into a format that can be read into a computer program for analysis.  For this project, I'll be using the [Python programming language](https://www.python.org/), and a few libraries (basically groupings of code that completes specialized common processes), most notably the [Natural Language Processing Toolkit (NLTK)](http://www.nltk.org/).

To do this, I am going to read 7 files in the .txt format, each containing text of one of the books, using Python's built-in `open` function and `read` method.  You can get the files for these and other books from [this site](https://archive.org/stream/pdfy-ZhGUmtnn6LEtA7jL/Harry%20Potter%20and%20the%20Philosopher%27s%20Stone%2C%20by%20J.K.%20Rowling_djvu.txt) -- note that you can use existing txt format files, or copy text, paste it in Notepad and save as a .txt file.

In this code, the files are named 'hp' then the book number - i.e. `hp1.txt` - and stored in a folder called `corpus`. To run the code as it is, you'll need to recreate this schema or alter the code to fit the path you create.

Let's write a function to read in our text.  

In [1]:
def read_file(num):
    text = ''
    with open('corpus/hp'+ str(num) + '.txt', 'rt') as file_in:
        for line in file_in:
            text = text + line
    return text

book_content = read_file(6)
print(book_content[3000:4002])

For a brief moment he allowed himself the impossible hope that nobody would answer him. However, a voice responded at once, a crisp, decisive voice that sounded as though it were reading a prepared statement. It was coming -- as the Prime Minister had known at the first cough -- from the froglike little man wearing a long silver wig who was depicted in a small, dirty oil painting in the far corner of the room.
"To the Prime Minister of Muggles. Urgent we meet. Kindly respond immediately. Sincerely, Fudge."
The man in the painting looked inquiringly at the Prime Minister.
"Er," said the Prime Minister, "listen... It's not a very good time for me... I'm waiting for a telephone call, you see... from the President of--"
"That can be rearranged," said the portrait at once. The Prime Minister's heart sank. He had been afraid of that.
"But I really was rather hoping to speak--"
"We shall arrange for the President to forget to call. He will telephone tomorrow night instead," said the little ma

Running this would return a single long string of text, and it's now usable in our program.  Notice that we pass in the number of the book we want to open with the `num` variable.

#### Step 2: Seperate dialog from narrative
In the text above, the opening and closing quotes look the same: `"`.  However, when we tokenize the text, NLTK makes opening and closing quotes look different, so we can see where dialog begins and ends.  Let's give it a try!

In [2]:
from nltk import word_tokenize

def tokenize_text(book_content):
    tokenized = word_tokenize(book_content)
    return tokenized

tokenized = tokenize_text(book_content)
print (tokenized[604:822])

['For', 'a', 'brief', 'moment', 'he', 'allowed', 'himself', 'the', 'impossible', 'hope', 'that', 'nobody', 'would', 'answer', 'him', '.', 'However', ',', 'a', 'voice', 'responded', 'at', 'once', ',', 'a', 'crisp', ',', 'decisive', 'voice', 'that', 'sounded', 'as', 'though', 'it', 'were', 'reading', 'a', 'prepared', 'statement', '.', 'It', 'was', 'coming', '--', 'as', 'the', 'Prime', 'Minister', 'had', 'known', 'at', 'the', 'first', 'cough', '--', 'from', 'the', 'froglike', 'little', 'man', 'wearing', 'a', 'long', 'silver', 'wig', 'who', 'was', 'depicted', 'in', 'a', 'small', ',', 'dirty', 'oil', 'painting', 'in', 'the', 'far', 'corner', 'of', 'the', 'room', '.', '``', 'To', 'the', 'Prime', 'Minister', 'of', 'Muggles', '.', 'Urgent', 'we', 'meet', '.', 'Kindly', 'respond', 'immediately', '.', 'Sincerely', ',', 'Fudge', '.', "''", 'The', 'man', 'in', 'the', 'painting', 'looked', 'inquiringly', 'at', 'the', 'Prime', 'Minister', '.', '``', 'Er', ',', "''", 'said', 'the', 'Prime', 'Minister

As you can see in the text above, our opening and closing quotes look like this: `'``' and "''"`.  This is a helpful tool in judging where dialog begins and ends across sentences.  Next, we will set up some rules around this and label parts of text.

#### Step 3: Label dialog and narration
In this step, we will label these two types of text while keeping them in order in case we need context.  To do this, we'll keep the list format to preserve order and create a tuple for each piece of dialogue.  So, for example:

`"''", 'That', 'can', 'be', 'rearranged', ',', "''", 'said', 'the', 'portrait', 'at', 'once', '.'`

Would become:

`('d', ["''", 'That', 'can', 'be', 'rearranged', ',', "''",]), ('n', ['said', 'the', 'portrait', 'at', 'once', '.'])`

What we'll do to achieve this is:
1. Create a new list called `parsed`.
1. Loop through the text in `tokenized` variable (printed above).  When we hit an open quote character, stop, grab everything up to that point and make it a list in a tuple where the first value is `n` for "narration", and the second value is a list containing all of those words (e.g. `('n', ['For', 'a', 'brief', 'moment', 'he', 'allowed', 'himself', 'the', 'impossible', 'hope', 'that', 'nobody', 'would', 'answer', 'him', '.', 'However', ',', 'a', 'voice', 'responded', 'at', 'once', ',', 'a', 'crisp', ',', 'decisive', 'voice', 'that', 'sounded', 'as', 'though', 'it', 'were', 'reading', 'a', 'prepared', 'statement', '.', 'It', 'was', 'coming', '--', 'as', 'the', 'Prime', 'Minister', 'had', 'known', 'at', 'the', 'first', 'cough', '--', 'from', 'the', 'froglike', 'little', 'man', 'wearing', 'a', 'long', 'silver', 'wig', 'who', 'was', 'depicted', 'in', 'a', 'small', ',', 'dirty', 'oil', 'painting', 'in', 'the', 'far', 'corner', 'of', 'the', 'room', '.',']`.  
2. Append this tuple to `text_parsed`.
3. Use the point where we found the open quote as a placeholder, then look ahead until we find a close quote.
4. Take the whole slice, from open quote to close quote.
4. Drop the slice into a tuple where the first value is `d` for "dialog" and the second value is a list containing all of the words and the quotes (e.g. `('d', '``', 'To', 'the', 'Prime', 'Minister', 'of', 'Muggles', '.', 'Urgent', 'we', 'meet', '.', 'Kindly', 'respond', 'immediately', '.', 'Sincerely', ',', 'Fudge', '.', "''")`)

The function to do this will be called `parse_text`. 

In [42]:
def parse_text(t):
    open_q = '``'
    close_q = "''"
    found_c = False # this will be used to break the while loop below
    # current will hold words until an open quote is found
    current = []
    # parsed is the list we'll eventually return, and where the ('n', ['sentence']) or ('d', ['sentence']) tuples
    # will be appended
    parsed = [] 
    length = len(t)
    i = 0

    while i < length:
        word = t[i]
        
        if word != open_q and word != close_q:
            current.append(word)

        elif word == open_q or word == close_q:
            parsed.append(('n', current))
            
            current = []
            current.append(word)
            
            while found_c == False and i < length-1:
                i += 1
                if t[i] != close_q:
                    current.append(t[i])
                else:
                    current.append(t[i])
                    parsed.append(('d', current))
                    current = []
                    found_c = True
        
        found_c = False
        i += 1
        
    return parsed
        

In [43]:
parsed = parse_text(tokenized)

# the text below is the same as above, now categorized with an 'n' (narration) or 'd' (dialog)
print(parsed[:13])

[('n', ['Chapter', '1', ':', 'The', 'Other', 'Minister', 'It', 'was', 'nearing', 'midnight', 'and', 'the', 'Prime', 'Minister', 'was', 'sitting', 'alone', 'in', 'his', 'office', ',', 'reading', 'a', 'long', 'memo', 'that', 'was', 'slipping', 'through', 'his', 'brain', 'without', 'leaving', 'the', 'slightest', 'trace', 'of', 'meaning', 'behind', '.', 'He', 'was', 'waiting', 'for', 'a', 'call', 'from', 'the', 'President', 'of', 'a', 'far', 'distant', 'country', ',', 'and', 'between', 'wondering', 'when', 'the', 'wretched', 'man', 'would', 'telephone', ',', 'and', 'trying', 'to', 'suppress', 'unpleasant', 'memories', 'of', 'what', 'had', 'been', 'a', 'very', 'long', ',', 'tiring', ',', 'and', 'difficult', 'week', ',', 'there', 'was', 'not', 'much', 'space', 'in', 'his', 'head', 'for', 'anything', 'else', '.', 'The', 'more', 'he', 'attempted', 'to', 'focus', 'on', 'the', 'print', 'on', 'the', 'page', 'before', 'him', ',', 'the', 'more', 'clearly', 'the', 'Prime', 'Minister', 'could', 'see'

#### Step 4: Distinguish parts of speech, like nouns, verbs, and adjectives
Now that we've categorized text as narration or dialog, we can use NLTK's classification function to categorize words by the part of speech they represent.

The `pos_tag` function of NLTK that we'll use to do this takes in a word as a string and returns a tuple of the word and a code for how it was classified.  For example, 'NNP' means the word has been tagged by NLTK as a proper noun, 'VB' means the word has been tagged as a verb.  So "Harry" in the sentence "Harry is Petunia's nephew." would come back as `('Harry', 'NNP')`.  

You can find a full list [here](https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/), but in the next steps we'll be focused primarily on adjectives, verbs and nouns.

We will import the `pos_tag` function from NLTK and tag the words in a new function called `tagged_text`.

In [44]:
from nltk import pos_tag

def tagged_text(t):
    tagged = [(tuple[0], [pos_tag(tuple[1])]) for tuple in t]
    return tagged

In [45]:
# we'll pass the function the parsed variable in which the text is classified as narration or dialog
# and then print the same text as above with the tagging

tagged = tagged_text(parsed)
print(tagged[:13])

[('n', [[('Chapter', 'NN'), ('1', 'CD'), (':', ':'), ('The', 'DT'), ('Other', 'JJ'), ('Minister', 'NNP'), ('It', 'PRP'), ('was', 'VBD'), ('nearing', 'VBG'), ('midnight', 'NN'), ('and', 'CC'), ('the', 'DT'), ('Prime', 'NNP'), ('Minister', 'NNP'), ('was', 'VBD'), ('sitting', 'VBG'), ('alone', 'RB'), ('in', 'IN'), ('his', 'PRP$'), ('office', 'NN'), (',', ','), ('reading', 'VBG'), ('a', 'DT'), ('long', 'JJ'), ('memo', 'NN'), ('that', 'WDT'), ('was', 'VBD'), ('slipping', 'VBG'), ('through', 'IN'), ('his', 'PRP$'), ('brain', 'NN'), ('without', 'IN'), ('leaving', 'VBG'), ('the', 'DT'), ('slightest', 'JJS'), ('trace', 'NN'), ('of', 'IN'), ('meaning', 'VBG'), ('behind', 'NN'), ('.', '.'), ('He', 'PRP'), ('was', 'VBD'), ('waiting', 'VBG'), ('for', 'IN'), ('a', 'DT'), ('call', 'NN'), ('from', 'IN'), ('the', 'DT'), ('President', 'NNP'), ('of', 'IN'), ('a', 'DT'), ('far', 'RB'), ('distant', 'JJ'), ('country', 'NN'), (',', ','), ('and', 'CC'), ('between', 'IN'), ('wondering', 'VBG'), ('when', 'WRB')

To recap: now we have a big list of tuples that represents an entire book, with the text still in its original order. 

Each item in the list is a tuple.  

The first value is the tuple an `'n'` for narration or `'d'` for dialog.  

The second item in the tuple is a list of tuples.  Each of these tuples include a word or punctuation mark and the appropriate tag for the part of speech it represents.

In [ ]:
#### Step 4: Teasing out sexist language


#### Sexist language
Since sexism in language is sometimes very subtle, a challenge for this project is identifying what words or combinations of words and their context should be considered biased.

After some searching, I found a couple of excellent lists.  Because the Harry Potter series is written in English by a British writer, I focused on sources from the UK and countries in the Commonwealth.  Using this blog post by a [New Zealand blogger](http://sacraparental.com/2016/05/14/everyday-misogyny-122-subtly-sexist-words-women/) I had a first set of words and some excellent categories to begin with.  I found a number of [other](http://time.com/4268325/history-calling-women-shrill/) excellent articles about sexism in language, which I used to add to the `sexist_words` Python dictionary below.

In [ ]:
sexist_words ={ 
    'assertiveness': ['bossy', 'abrasive', 'ball-buster', 'aggressive', 'shrill', 'bolshy', 'intense', 'stroppy', 'forward', 'mannish', 'strident', 'know-it-all'],
    'behavior' : ['cackle', 'shriek', 'caterwaul', 'yowl', 'screech','gossip', 'dramatic', 'catty', 'bitch', 'nag', 'cold', 'icy', 'shrew', 'humorless', 'man-hater', 'banshee', 'fishwife', 'lippy', 'ditzy', 'diva', 'prima donna', 'feisty', 'ladylike', 'bubbly', 'vivaious', 'flirt', 'sass', 'chatty', 'demure', 'modest', 'emotional', 'hysterical', 'hormonal', 'menstrual', 'flaky', 'moody', 'over-sensitive'],
    'sexuality': ['slut', 'trollop', 'frigid', 'easy', 'tease', 'loose', 'man-eater', 'prude', 'curvy', 'cheap', 'frumpy', 'faded', 'mousey', 'clotheshorse', 'cow', 'hag'],
    'relationship': ['spinster', 'barren', 'housewife', 'houseproud', 'soccer mom', 'mistress', 'kept woman'],
    'praise': ['caring', 'compassionate', 'hard-working', 'conscientious', 'dependable', 'diligent', 'dedicated', 'tactful', 'interpersonal', 'warm', 'helpful'],
}

#### Step 3: Parsing the text
